# A look at ext4

In this IPython notebook I want to give an interactive "hello world" walkthrough of how data is organized within ext4, hoping that in the process I will also find out a bit about how it works.

## Creating up the filesystem

One can create an ext4 filesystem on a file (which can also be a block device, such as hard drive) using `mkfs.ext4` command. Let's write some boilerplate code for calling Ubuntu commands, create such device and take a look at what's there:

In [ ]:
from subprocess import Popen, STDOUT
C = lambda *a, **kw: print(
    check_output(
        *a, **kw, stderr=STDOUT, shell=True
    ).decode('ascii'))

# Let's create an empty file that is 128 kibibytes large, then mkfs.ext4 it:
C("dd if=/dev/zero bs=1024 count=128 of=fs.img && mkfs.ext4 fs.img")

As we can see, we now have a file that once contained 131072 zeros and now has an ext4 layout initialized there. It's probably worth noting that we got a warning that the filesystem is `too small for a journal`, which means tha at least one of the features was disabled.